<a href="https://colab.research.google.com/github/FabriceBeaumont/1-MA-INF_2314_IPSA_Repo/blob/main/Code/Exercise4/SolutionsOfSheet4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPSA 2021 - Exercise 4 - Fourier Transforms and Gaussian filters

## 0 - Practical Advice

In [4]:
import imageio 
import numpy as np
import numpy.fft as fft
import numpy.random as rnd
import scipy.ndimage as img
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 6]
import timeit, functools

In [5]:
def imageScaleRange(f, vmin=0, vmax=255):
    """Stretches the intensity values of a given image array f 
    such that they cover the whole range from 0 to 255"""
    return np.interp(f, (f.min(), f.max()), (vmin, vmax))

def imageRead(imgname, pilmode ='L', arrtype=float):
    """
    Read an image file into a numpy array

    imgname: str
        name of image file to be read
    pilmode: str
        for luminance / intesity images use ’L’
        for RGB color images use ’RGB’
    
    arrtype: numpy dtype
        use np.float, np.uint8, ...
    """
    return imageio.imread(imgname, pilmode=pilmode).astype(arrtype)

def imageWrite(arrF, imgname, arrtype=np.uint8):
    """
    Write a numpy array as an image file 
    the file type is inferred from the suffix of parameter imgname, e.g. ’.png’
    arrF: array_like
        array to be written
    imgname: str
        name of image file to be written
    arrtype: numpy dtype
        use np.uint8, ...
    """
    imageio.imwrite(imgname, arrF.astype(arrtype))

In [6]:
def writeFigure(fig, fname):
    fmt = fname.split('.')[-1]
    fig.savefig(fname, facecolor=fig.get_facecolor(), edgecolor='w',
                papertype='letter', format=fmt, transparent=False,
                bbox_inches='tight', pad_inches=0)

def plotFcts(x, fs,
             labs=None, lfs='x-small',
             xtcs=None, ytcs=None,
             lims=None, fname=None, title=None):

    # Initialize figure and axes
    fig = plt.figure()
    axs = fig.add_subplot(211, facecolor='w')

    # Nicer way of showing coordinate axes
    for pos in ['left','bottom']:
        axs.spines[pos].set_position('zero')
    for pos in ['right','top']:
        axs.spines[pos].set_visible(False)
        
    # Plot the function
    for f in fs:
        axs.plot(x, f, '-', alpha=0.75)

    # Plot the legend
    if labs is not None:
        axs.legend(labs, loc="upper right", fontsize=lfs,
                   facecolor='#e0e0e0', edgecolor='k', fancybox=False)

    if xtcs is not None: axs.set_xticks(xtcs)
    if ytcs is not None: axs.set_yticks(ytcs)
    
    if lims is not None:
        axs.set_xlim(lims[0], lims[1])
        axs.set_ylim(lims[2], lims[3])
        
    if title:
        plt.title(title)
        
        
    # Show figure on screen or write it to disc
    plt.show() if fname is None else writeFigure(fig, fname)
    plt.close()

## 1 - Plotting boundaries of shape images

In the Data folder for this exercise, you will find the data file `hand.csv` which contains a data matrix $X\in\mathbb{R}^{2\times 537}$ whose columns $x_k = [x_k , y_k ]^\intercal$ represent 2D points. If you plot these data points using individual dots or a closed line.

![dot_and_line.png](attachment:dot_and_line.png)

To read these data into a numpy array `matX` representing matrix $X$, you may proceed as follows:

In [7]:
matX = np.loadtxt('hand.csv', delimiter=', ')

Now, center the data in $X$, i.e. determine the the mean $\hat{x}$ of the data points $x_k$ and transform them as follows:
$$  x_k  \leftarrow \quad x_k - \hat{x} $$


Create a dot plot and a line plot of the centered data (including coordinate axes).

## 2 - Interpolating boundaries of shape images

In what follows, we will work with the centered data matrix $\hat{X}$ which you computed in the previous task and we will assume that is  available in a 2D numpy array `matXh`.

Recall that this matrix contains $n = 537$ column vectors $x_k = [x_k , y_k ]^\intercal$.
Given these $n$ data points, we will now interpolate $N$. Without further ado, this can be accomplished as follows:

In [10]:
matXh = np.hstack((matXh, matXh[:,0].reshape(2,1)))
stps = np.linspace(0, 2*np.pi, matXh.shape[1])
iplx = ipl.interp1d(stps, matXh[0], kind='cubic')
iply = ipl.interp1d(stps, matXh[1], kind='cubic')

matXi = np.vstack((iplx(np.linspace(0, 2*np.pi, N, endpoint=False)), iply(np.linspace(0, 2*np.pi, N, endpoint=False))))

NameError: ignored

Create plots dot- and line for the cases where for $N = \{256, 512\}$.

## 3 - Fourier transforms of periodic complex valued functions

Note that we can alternatively think of the 2D data points $x_k = [x_k , y_k ]^\intercal$ from the previous tasks in terms of complex numbers
$$ z_k = x_k + i y_k $$

Also note that we can therefore think of our use of `ipl.interp1d` in the previous task as having created a continuous function $f:[0, 2\pi ) \to C$ which we then sampled at points $j · \frac{2\pi}{N}$ where $j = 0, \dots , N − 1$ to obtain a vector or 1D array 
$$ z = [ z_0, z_1 , \dots z_{N-1}] $$

where $z_j = f\big( j\frac{2\pi}{N} \big)$.

Finally, note that we may think of array $z$ as the values of one period of the discrete periodic function $f[j] = f \big( j\frac{2\pi}{N} \big)$ where $j = J \text{mod} N$ and $J\in\mathbb{Z}$.

To compute array $z$ in practice, i.e. not just conceptually, you may reuse your results from the previous task. Letting $N = 256$, recompute array `arrXi` and then execute

In [ ]:
arrZ = matXi[0] + 1j * matXi[1]

Now compute the Fourier transform of array $z$ and low pass filter it with cut-off frequencies $\omega_l\in \{2, 4\}$. (Remember our discussion of `fft.fftfreq` in *lecture 10* and of `fft.fftshift` and `fft.ifftshift` in *lecture 11*.)

Compute the inverse Fourier transform of the filtered signal and create line plots of your results. To see how these should look like, here are results for $\omega_l\in \{0.5, 1.0\}$.

## 4 Gaussian filtering in practice

In the Data folder for this exercise, you will find the intensity image `cat.png`. Read it into a numpy array $arrF$.

In the lecture we discussed Gaussian filtering in the space domain and saw that we can think of the process as moving a corresponding filter
mask across the image to be filtered. Back then, the question arose how to handle pixels close to the boundary of the image and your instructor
said not to obsess about this issue...

Indeed, Gaussian filtering is such a common task that `scipy` has us covered. Its `ndimage` module contains the function gaussian filter which you may use like this:

In [11]:
sigm = 7.
arrH = img.gaussian_filter(arrF, sigma=sigm, mode='constant')

SyntaxError: ignored

Note, however, that constant is not the only possible value for parameter mode. This parameter tell the function how to handle the “boundary pixel problem” and if you read the scipy manual, you will find that it can also be
set to reflect, mirror, nearest,...

Consider $\sigma \in \{1, 3, 9\}$ and the mode parameters just mentioned to compute filtered images $arrH$. Write these images as PNG files.

What do you observe? Does the manner in which images boundaries are handled make a huge difference in practice?